## Bigdl-nao onnxruntime example
--- 
This example shows the usage of bigdl-nano pytorch onnxtuntime. 

In [4]:
import os
from time import time

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from pl_bolts.datamodules import CIFAR10DataModule
from pl_bolts.transforms.dataset_normalizations import cifar10_normalization
from pytorch_lightning import LightningModule, seed_everything
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from pytorch_lightning.loggers import TensorBoardLogger
from torch.optim.lr_scheduler import OneCycleLR
from torchmetrics.functional import accuracy
from bigdl.nano.pytorch.trainer import Trainer
from bigdl.nano.pytorch.vision import transforms
import numpy as np
import pdb

### CIFAR10 Data Module
---
Import the existing data module from bolts and modify the train and test transforms.
You could access [CIFAR10](https://www.cs.toronto.edu/~kriz/cifar.html) for a view of the whole dataset.

In [5]:
def prepare_data(data_path, batch_size, num_workers):
    train_transforms = transforms.Compose(
        [
            transforms.RandomCrop(32, 4),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            cifar10_normalization()
        ]
    )

    test_transforms = transforms.Compose(
        [
            transforms.ToTensor(),
            cifar10_normalization()
        ]
    )
    cifar10_dm = CIFAR10DataModule(
        data_dir=data_path,
        batch_size=batch_size,
        num_workers=num_workers,
        train_transforms=train_transforms,
        test_transforms=test_transforms,
        val_transforms=test_transforms
    )
    return cifar10_dm

### Resnet
___
Modify the pre-existing Resnet architecture from TorchVision. The pre-existing architecture is based on ImageNet images (224x224) as input. So we need to modify it for CIFAR10 images (32x32).

In [6]:
def create_model():
    model = torchvision.models.resnet18(pretrained=False, num_classes=10)
    model.conv1 = nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    model.maxpool = nn.Identity()
    return model

### Lightning Module
___
Check out the [configure_optimizers](https://pytorch-lightning.readthedocs.io/en/stable/common/lightning_module.html#configure-optimizers) method to use custom Learning Rate schedulers. The OneCycleLR with SGD will get you to around 92-93% accuracy in 20-30 epochs and 93-94% accuracy in 40-50 epochs. Feel free to experiment with different LR schedules from https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate

In [7]:
class LitResnet(LightningModule):

    def __init__(self, learning_rate=0.05):
        super().__init__()

        self.save_hyperparameters()
        self.model = create_model()
        self.example_input_array = torch.Tensor(64, 3, 32, 32)

    def forward(self, x):
        out = self.model(x)
        return F.log_softmax(out, dim=1)

    def training_step(self, batch, batch_idx):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        self.log("train_loss", loss)
        return loss

    def evaluate(self, batch, stage=None):
        x, y = batch
        logits = self(x)
        loss = F.nll_loss(logits, y)
        preds = torch.argmax(logits, dim=1)
        acc = accuracy(preds, y)

        if stage:
            self.log(f"{stage}_loss", loss, prog_bar=True)
            self.log(f"{stage}_acc", acc, prog_bar=True)

    def validation_step(self, batch, batch_idx):
        self.evaluate(batch, "val")

    def test_step(self, batch, batch_idx):
        self.evaluate(batch, "test")

    def configure_optimizers(self):
        optimizer = torch.optim.SGD(
            self.parameters(),
            lr=self.hparams.learning_rate,
            momentum=0.9,
            weight_decay=5e-4,
        )
        steps_per_epoch = 45000
        scheduler_dict = {
            "scheduler": OneCycleLR(
                optimizer,
                0.1,
                epochs=self.trainer.max_epochs,
                steps_per_epoch=steps_per_epoch,
            ),
            "interval": "step",
        }
        return {"optimizer": optimizer, "lr_scheduler": scheduler_dict}

In [8]:
seed_everything(7)
PATH_DATASETS = os.environ.get("PATH_DATASETS", ".")
BATCH_SIZE = 64
NUM_WORKERS = int(os.cpu_count() / 2)
data_module = prepare_data(PATH_DATASETS, BATCH_SIZE, NUM_WORKERS)

Global seed set to 7


In [9]:
pl_model = LitResnet(learning_rate=0.05)
pl_model.datamodule = data_module
trainer = Trainer(num_processes = 1,
                  use_ipex = False,
                  progress_bar_refresh_rate=10,
                  max_epochs=30,
                  logger=TensorBoardLogger("lightning_logs/", name="basic"),
                  callbacks=[LearningRateMonitor(logging_interval="step")])
trainer.fit(model, dm)
trainer.test(model, dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs


NameError: name 'model' is not defined

In [10]:
pl_model = LitResnet.load_from_checkpoint("lightning_logs/resnet.ckpt", learning_rate=0.05)
data_module.setup("test")

In [11]:
start = time()
pl_model.eval()
for x, _ in data_module.test_dataloader():
    with torch.no_grad():
        pl_model(x)
infer_time = time() - start
outputs = trainer.test(pl_model, datamodule=data_module)
fp32_acc = outputs[0]['test_acc'] * 100

Files already downloaded and verified
Files already downloaded and verified


/home/mingzhi/anaconda3/envs/forOpenvino/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(


Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.8615000247955322, 'test_loss': 0.4319942891597748}
--------------------------------------------------------------------------------


### Get Accelerated Module
---
Use Train.trace from bigdl.nano.pytorch.trainer to convert a model into an accelerated module for inference.
The definition of trace is:
```
trace(model: nn.Module, input_sample=None, accelerator=None)

      :param model: An torch.nn.Module model, including pl.LightningModule.
      
      :param input_sample: A set of inputs for trace, defaults to None if you have trace before or
                             model is a LightningModule with an example_input_array.
                             
      :param accelerator: The accelerator to use, defaults to None meaning staying in Pytorch
                            backend. 'openvino' and 'onnxruntime' are supported for now.
                            
      :return: Model with different acceleration(OpenVINO/ONNX Runtime).
```
- *Note* <br>
trace is a class method. You should use your Trainer class to call it instead of the Trainer instance.

In [12]:
onnx_model = Trainer.trace(pl_model, accelerator="onnxruntime")
start = time()
for x, _ in pred_loader:
    inference_res_onnx = onnx_model(x)
onnx_infer_time = time() - start

In [13]:
openvino_model = Trainer.trace(pl_model, accelerator="openvino")
start = time()
for x, _ in data_module.test_dataloader():
    inference_res_openvino = openvino_model(x)
openvino_infer_time = time() - start

Model Optimizer arguments:
Common parameters:
	- Path to the Input Model: 	/home/mingzhi/PycharmProjects/plExample/src/nanOnnx/tmp.onnx
	- Path for generated IR: 	/home/mingzhi/PycharmProjects/plExample/src/nanOnnx/.
	- IR output name: 	tmp
	- Log level: 	ERROR
	- Batch: 	Not specified, inherited from the model
	- Input layers: 	Not specified, inherited from the model
	- Output layers: 	Not specified, inherited from the model
	- Input shapes: 	Not specified, inherited from the model
	- Source layout: 	Not specified
	- Target layout: 	Not specified
	- Layout: 	Not specified
	- Mean values: 	Not specified
	- Scale values: 	Not specified
	- Scale factor: 	Not specified
	- Precision of IR: 	FP32
	- Enable fusing: 	True
	- User transformations: 	Not specified
	- Reverse input channels: 	False
	- Enable IR generation for fixed input shape: 	False
	- Use the transformations config file: 	None
Advanced parameters:
	- Force the usage of legacy Frontend of Model Optimizer for model conversion in

In [14]:
template = """
|    Precision   | Inference Time(s) |
|     Pytorch    |       {:5.2f}       |
|      ONNX      |       {:5.2f}       |
|    Openvino    |       {:5.2f}       |
"""
summary = template.format(
    
    infer_time,
    onnx_infer_time,
    openvino_infer_time
)
print(summary)


|    Precision   | Inference Time(s) |
|     Pytorch    |       62.45       |
|      ONNX      |       30.42       |
|    Openvino    |       150.36       |



### Calibrate Model
Use Trainer.quantize from bigdl.nano.pytorch.trainer to calibrate a Pytorch-Lightning model for post-training quantization. Here are some important paramters:
```
:param pl_model:         A Pytorch-Lightning model to be quantized.
:param calib_dataloader:         A torch.utils.data.dataloader.DataLoader object for calibration.     
                                 Required for static quantization.
:param approach:         'static' or 'dynamic'.
                         'static': post_training_static_quant,
                         'dynamic': post_training_dynamic_quant.
                          Default: 'static'.

```
Access more details from [Source](https://github.com/intel-analytics/BigDL/blob/main/python/nano/src/bigdl/nano/pytorch/trainer/Trainer.py)

In [15]:
i8_model = trainer.quantize(pl_model, calib_dataloader=data_module.test_dataloader())
start = time()
for x, _ in data_module.test_dataloader():
    inference_res_i8 = i8_model(x)
i8_inference_time = time() - start
outputs = trainer.test(i8_model, datamodule=data_module)
i8_acc = outputs[0]['test_acc'] * 100

2022-05-10 12:10:19 [INFO] Generate a fake evaluation function.
2022-05-10 12:10:19 [INFO] Pass query framework capability elapsed time: 86.9 ms
2022-05-10 12:10:19 [INFO] Get FP32 model baseline.
2022-05-10 12:10:19 [INFO] Save tuning history to /home/mingzhi/PycharmProjects/plExample/src/nanOnnx/nc_workspace/2022-05-10_12-10-18/./history.snapshot.
2022-05-10 12:10:19 [INFO] FP32 baseline is: [Accuracy: 1.0000, Duration (seconds): 0.0000]
/home/mingzhi/anaconda3/envs/forOpenvino/lib/python3.9/site-packages/torch/quantization/observer.py:122: UserWarning: Please use quant_min and quant_max to specify the range for observers.                     reduce_range will be deprecated in a future release of PyTorch.
  warnings.warn(
2022-05-10 12:10:19 [WARNING] Please note that calibration sampling size 100 isn't divisible exactly by batch size 64. So the real sampling size is 128.
/home/mingzhi/anaconda3/envs/forOpenvino/lib/python3.9/site-packages/torch/_tensor.py:575: UserWarning: floor_div

Testing: 0it [00:00, ?it/s]

--------------------------------------------------------------------------------
DATALOADER:0 TEST RESULTS
{'test_acc': 0.8615000247955322, 'test_loss': 0.4279630482196808}
--------------------------------------------------------------------------------


/home/mingzhi/anaconda3/envs/forOpenvino/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:423: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(


In [20]:
template = """
|    Precision   | Inference Time(s) | Accuracy(%) |
|      FP32      |       {:5.2f}       |    {:5.2f}    |
|      INT8      |       50.34       |    {:5.2f}    |
| Improvement(%) |       19.       |    {:5.2f}    |
"""
summary = template.format(
    infer_time, fp32_acc,
     i8_acc,
    (1 - i8_inference_time /infer_time) * 100,
    i8_acc - fp32_acc
)
print(summary)


|    Precision   | Inference Time(s) | Accuracy(%) |
|      FP32      |       62.45       |    86.15    |
|      INT8      |       59.34       |    86.15    |
| Improvement(%) |       -7.72       |     0.00    |

